In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx
import random
import pickle
import itertools
import urllib.request as urllib
import io
import zipfile
import re
import time
import datetime 
import dynetx as dn
import copy
import ast
import glob
import scipy.io

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras import layers
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.linalg import diag
from tensorflow.keras import callbacks
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score
from networkx.generators.community import LFR_benchmark_graph
from itertools import count
from rdyn import RDyn
from datetime import datetime

from helpers import *
from Autoencoder import *

In [ ]:
#KARATE CLUB
kc = nx.karate_club_graph()
pickle.dump(kc, open("kc.p", "wb"))

In [ ]:
#FOOTBALL
url = "http://www-personal.umich.edu/~mejn/netdata/football.zip"

sock = urllib.urlopen(url)  
s = io.BytesIO(sock.read())  
sock.close()

zf = zipfile.ZipFile(s) 
txt = zf.read("football.txt").decode() 
gml = zf.read("football.gml").decode()  
gml = gml.split("\n")[1:]
football = nx.parse_gml(gml)  

pickle.dump(football, open("football.p", "wb"))

In [ ]:
#LFR with mu=0.1
lfr_1 = generate_LFR_network(1000, 0.1, 20, 100, 50, 100, 1000)
pickle.dump(lfr_1, open("lfr_1.p", "wb"))

In [ ]:
#LFR with mu=0.3
lfr_2 = generate_LFR_network(1000, 0.3, 20, 100, 50, 100, 1000)
pickle.dump(lfr_2, open("lfr_2.p", "wb"))

In [ ]:
#LFR with mu=0.5
lfr_3 = generate_LFR_network(1000, 0.5, 20, 100, 50, 100, 1000)
pickle.dump(lfr_3, open("lfr_3.p", "wb"))

In [ ]:
#LFR with mu=0.7
lfr_4 = generate_LFR_network(1000, 0.7, 20, 100, 50, 100, 1000)
pickle.dump(lfr_4, open("lfr_4.p", "wb"))

In [69]:
#CORA
cora_cites = pd.read_csv('graphs/cora_cites.csv')

cora_features = pd.read_csv('graphs/cora_features.csv')
cora_features['word_cited_id'] = cora_features['word_cited_id'].map(lambda x: x.lstrip('word'))
cora_features['word_cited_id'] = cora_features['word_cited_id'].astype('int32')

cora_labels = pd.read_csv('graphs/cora_labels.csv')

In [70]:
cora = nx.OrderedGraph()

for index, row in cora_cites.iterrows():
    p1 = row['cited_paper_id']
    p2 = row['citing_paper_id']
    c1 = list(cora_labels[cora_labels['paper_id'] == p1]['class_label'])[0]
    c2 = list(cora_labels[cora_labels['paper_id'] == p2]['class_label'])[0]
    f1 = {'gt': c1}
    f2 = {'gt': c2}
    d1 = dict.fromkeys(list(cora_features[cora_features['paper_id'] == p1]['word_cited_id']), 1)
    d2 = dict.fromkeys(list(cora_features[cora_features['paper_id'] == p2]['word_cited_id']), 1)
    
    for i in range(1433):
        f1[i] = d1.get(i+1, 0)
        f2[i] = d2.get(i+1, 0)
        
    n1 = (p1, f1)
    n2 = (p2, f2)
    
    cora.add_nodes_from([n1, n2])
    cora.add_edge(p1, p2)

In [ ]:
pickle.dump(cora, open("cora.p", "wb"))

In [ ]:
#building citeseer graph
edges = pd.read_csv('graphs/citeseer-edges.csv')
group_edges = pd.read_csv('graphs/citeseer-group-edges.csv')
node_content = pd.read_csv('graphs/citeseer_node_features.csv', dtype=str)

cs = nx.OrderedGraph()
nodes = []
feat_dict = {}

for index, row in node_content.iterrows():
    el = row['Node']
    group = row['Label']
    membership = {'gt': group}
    features = row[1:len(node_content.columns)-1].tolist()
    node_feat_dict = {}
    for i in range(len(features)):
        node_feat_dict[i] = int(features[i])
    feat_dict[el] = node_feat_dict
    node = (el, membership)
    nodes.append(node)
    
cs.add_nodes_from(nodes)

for row in edges.iterrows():
    cs.add_edge(row[1][0], row[1][1])
    
nx.set_node_attributes(cs, feat_dict)

pickle.dump(cs, open("citeseer.p", "wb"))

In [2]:
#RDyn Synthetic Dynamic Graph
rdb = RDyn()
rdb.execute(simplified=True)

100%|█████████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  5.94it/s]


21

In [130]:
#parse generated graph

graphNum = 99;
direc = './results/1000_100_15_0.6_0.8_0.2_1/'
out_file = "./graphs/RDyn/" + "graph-" + str(graphNum) + ".p"

graph = nx.OrderedGraph();

gname = 'graph-' + str(graphNum) + '.txt'
cname = 'communities-' + str(1) + '.txt'

with open(direc + cname) as f:
    mylist = list(f)
    for line in mylist:
        sline = str(line)
        community = int(sline.split()[0])
        nodes = ast.literal_eval(sline.lstrip('0123456789.- ').strip())
        for node in nodes:
            graph.add_nodes_from([node], gt=community)
    f.close()
        
with open(direc + gname) as f:
    mylist = list(f)
    for line in mylist:
        vertices = line.split();
        v1 = int(vertices[0])
        v2 = int(vertices[1])
        graph.add_edges_from([(v1, v2)])
    f.close()
    
with open(out_file, "wb") as f:
    pickle.dump(graph, f)
    f.close()

In [2]:
#sociopatterns high school

#load csv
thiers_df = pd.read_csv('graphs/thiers_2012.csv')
#convert unix time to datetime
thiers_df['timestamp'] = pd.to_datetime(thiers_df['timestamp'],unit='s')

thiers_df['timestamp'] = thiers_df['timestamp'].apply(lambda dt: datetime(dt.year, dt.month, dt.day, dt.hour,30*(dt.minute // 30)))
#load metadata
thiers_metadata = pd.read_csv('graphs/thiers_2012_metadata.csv')
#index by datetime
thiers_df = thiers_df.set_index('timestamp')

thiers_df = thiers_df.dropna()
thiers_df = thiers_df.astype({'ID1': 'int32', 'ID2': 'int32'})

#create base graph
baseGraph = nx.OrderedGraph()
l1 = list(thiers_df.drop_duplicates(subset=['ID1'])[['ID1', 'C1']].to_records(index=False))
for el in l1:
    m = {'gt': el[1]}
    baseGraph.add_nodes_from([(el[0], m)])

In [5]:
#create graphs for specific school day
day = 19
intervals = list(thiers_df[thiers_df.index.day == day].index.unique())
graphlist = []

for interval in intervals:
    g = copy.deepcopy(baseGraph)
    for row in thiers_df[thiers_df.index == interval].iterrows():
        g.add_edges_from([(row[1][0], row[1][1])])
    graphlist.append(g)

In [22]:
pickle.dump(graphlist, open("thiers_day_19.p", "wb"))

In [7]:
#DANCer graph creation
path = './graphs/DANCer_spicy/'
out_path = "./graphs/DANCer_spicy_built/"

gnames = ['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9']

for gname in gnames:
    graph = nx.OrderedGraph()
    fname = gname + '.graph'
    out_file = out_path  + gname + ".p"
    vertices = False;
    edges = False;
    with open(path + fname) as f:
        mylist = list(f)
        for line in mylist:
            line = line.strip()
            if(line == '#'):
                continue;
            if(line == '# Vertices'):
                vertices = True;
                edges = False;
                continue;
            if(line == '# Edges'):
                edges = True;
                vertices = False;
                continue;
            tokens = line.split(';')
            if(vertices):
                v = int(tokens[0])
                atts = tokens[1].split('|')
                com = int(tokens[len(tokens)-1])
                node_dict = {'gt': com}
                count = 0
                for att in atts:
                    node_dict[count] = float(att)
                    count += 1
                graph.add_nodes_from([(v, node_dict)])
            if(edges):
                v1 = int(tokens[0])
                v2 = int(tokens[1])
                graph.add_edge(v1, v2)
        f.close()
    with open(out_file, "wb") as f:
        pickle.dump(graph, f)
        f.close()

In [2]:
#DynEx.mat
mat = scipy.io.loadmat('./graphs/DynEx.mat')

In [20]:
mat['Anew'][1]

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)